<a href="https://colab.research.google.com/github/getcontrol/maskrcnn-hacks/blob/main/Mask_RCNNv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mask RCNN Training on Skymaps**

Credits: Nancy Cai

##**Reference:**
1. [Github-Mask RCNN Model](https://github.com/matterport/Mask_RCNN)
2. [Article-Splash of Color: Instance Segmentation with Mask R-CNN and TensorFlow](https://engineering.matterport.com/splash-of-color-instance-segmentation-with-mask-r-cnn-and-tensorflow-7c761e238b46)
3. [Github-Mask R-CNN Training and Inference](https://github.com/akTwelve/cocosynth/blob/master/notebooks/train_mask_rcnn.ipynb)

##**Set Up**


In [ ]:
%cd /content/
!rm -r sample_data

In [ ]:
from google.colab import drive    # RUN WHEN TRAINED WEIGHTS GET UPLOADED TO THE GOOGLE DRIVE,  
drive.mount('/content/drive')     # so you can access files in google drive

##**GPU**


In [ ]:
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

##**Setting Up Folder Structure**

1.   Mask_RCNN
2.   data
3.   weights

In [ ]:
%%capture
!rm -rf sample_data
!mkdir data
!mkdir weights
!gdown https://drive.google.com/u/0/uc?id=1eToBctRfz2b83KgP25xRsJ2mZAeqYtHB -O ./data/data.zip
!unzip ./data/data.zip -d ./data

In [ ]:
!wget -P weights/ https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/matterport/Mask_RCNN 

In [ ]:
 %cd /content/Mask_RCNN
 !python setup.py install
 %cd /content/
 !ls #make sure root conrent listed

In [ ]:
!pip install tensorflow==1.15.0 
!pip uninstall keras-nightly -y
!pip install h5py==2.10.0 # resolve the issue AttributeError: 'str' object has no attribute 'decode' 

In [ ]:
!pip install keras==2.1.0 --force-reinstall --no-deps --no-cache-dir

In [ ]:
%tensorflow_version 1.x

##**Import libs & Model dependent libs**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import datetime
import numpy as np
import random
import math
import re
import time
import tensorflow as tf

# plot images
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

# process images
from PIL import Image
from collections import defaultdict

# load data
from pathlib import Path
import json
import glob

# process coco data
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

# Evaluate
import cv2
import skimage
from skimage.metrics import structural_similarity as ssim

In [ ]:
# Change directory into cloned repo
%cd /content/Mask_RCNN

# List repo contents
!ls

In [ ]:
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

##**Split Train and Validation**

In [ ]:
dataroot = Path("/content/data")

def get_filenames(directory):
  filenames = [file for file in directory.rglob("*") if file.is_file()]
  return filenames

inputs_train=get_filenames(dataroot/"imgs"/"train")
targets_train=get_filenames(dataroot/"masks"/"train")

inputs_valid=get_filenames(dataroot/"imgs"/"validation")
targets_valid=get_filenames(dataroot/"masks"/"validation")


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

**Plot Train & Masked Images**


In [ ]:
# code for displaying multiple images 
# create figure
import os
fig = plt.figure(figsize=(10, 7))
  
def plot_images(rows, columns):
  # reading images
  each_images = int(rows*columns/2)
  for i in range(1,each_images+1):
      # Adds subplots for train images
      input_full_path = inputs_train[i].as_uri()
      relative_path = 'file:'
      input_image = '/' + os.path.relpath(input_full_path, relative_path)
      trainImage = mpimg.imread(input_image)
      trainImageName = inputs_train[i].name
      fig.add_subplot(rows, columns, i)
      plt.imshow(trainImage)
      plt.axis('off')
      plt.title(f"train_{i}")

      # Adds suplots for target images
      target_full_path = targets_train[i].as_uri()
      target_image = '/' + os.path.relpath(target_full_path, relative_path)
      targetImage = mpimg.imread(target_image)
      targetImageName = targets_train[i].name
      fig.add_subplot(rows, columns, columns+i)
      plt.imshow(targetImage)
      plt.axis('off')
      plt.title(f"target_{i}")

plot_images(2,5)

In [ ]:
# Get colours used in the images to define our annotation function
%cd /content/data/masks/train/

im = Image.open(targets_train[4].name)
by_color = defaultdict(int)
for pixel in im.getdata(): 
  by_color[pixel] += 1
by_color


##**Create annotation files based on masked data**
You could run this once and download the annotations for model training because it took more than one hour to get annotation files. I have saved the annotations files to [Google Driver](https://drive.google.com/drive/folders/1_WUjJ3ArrJLSvzvLcTDeSVQhcg-Gr3IV)

Mask RCNN needs annotation files for training and because this dataset is public and do not have annotations files, we generated annotaions using python script. 

**Reference**
1. [Article-Create COCO Annotations from Scratch](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch) 
2. [Github-Image to COCO JSON Converter](https://github.com/chrise96/image-to-coco-json-converter) 



In [ ]:
%cd /content

In [ ]:
!git clone -l -s https://github.com/chrise96/image-to-coco-json-converter

# Change directory into cloned repo
%cd image-to-coco-json-converter

# List repo contents
!ls

In [ ]:
# Label ids of the dataset
category_ids = {
    "background": 0,
    "weed": 1
#    "plant": 2 comment out plant 
}

# Define which colors match which categories in the images
category_colors = {
    "(0, 0, 0)": 0, # background
    "(2, 2, 2)": 0,
    "(39, 39, 39)": 1, # weed
    "(78, 78, 78)": 0 # change plant to background because only want to detect weed
}

# Define the ids that are a multiplolygon. In our case: weed
multipolygon_ids = [1] #[1,2]

# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    for mask_image in glob.glob(maskpath + "*.png"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image)

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
            category_id = category_colors[color]

            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)

            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                                
                annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                annotations.append(annotation)
                annotation_id += 1
            else:
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id

In [ ]:
!touch /content/data/imgs/train/train.json
!touch /content/data/imgs/validation/validation.json
!touch /content/data/imgs/test/test.json

In [ ]:
from src.create_annotations import *

coco_format = get_coco_json_format()
for keyword in ["train","validation","test"]:
  mask_path = "/content/data/masks/{}/".format(keyword)
  annotation_path = "/content/data/imgs/{}/".format(keyword)
  print(mask_path, annotation_path)
        
  # Create category section
  coco_format["categories"] = create_category_annotation(category_ids)
    
  # Create images and annotations sections
  coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

  with open("{}/{}.json".format(annotation_path,keyword),"w") as outfile:
    json.dump(coco_format, outfile)
        
  print("Created %d annotations for images in folder: %s" % (annotation_cnt, annotation_path))

##**View Annotations**



In [ ]:
import IPython
import os
import json
import random
import numpy as np
import requests
from io import BytesIO
import base64
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

In [ ]:
# Load the dataset json
class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                                'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                               'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                               'magenta', 'sienna', 'maroon']
        
        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        json_file.close()
        
        self.process_info()
        self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()
            
        
    def display_info(self):
        print('Dataset Info:')
        print('=============')
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))
        
        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')

        
    def display_licenses(self):
        print('Licenses:')
        print('=========')
        
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')
        
    def display_categories(self):
        print('Categories:')
        print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(cat_id, self.categories[cat_id]['name']))
            print('')
    
    def display_image(self, image_id, show_polys=True, show_bbox=True, show_crowds=True, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
            
        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))
            
        else:
            image_path = os.path.join(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)
            
            buffer = BytesIO()
            image.save(buffer, format='PNG')
            buffer.seek(0)
            
            data_uri = base64.b64encode(buffer.read()).decode('ascii')
            image_path = "data:image/png;base64,{0}".format(data_uri)
            
        # Calculate the size and adjusted display size
        max_width = 600
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        
        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})
                            
                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})
                                
                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list  
            else:
                # Add the polygon segmentation
                for segmentation_points in segm['segmentation']:
                    segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
                    polygons_list.append(str(segmentation_points).lstrip('[').rstrip(']'))
            polygons[segm['id']] = polygons_list
            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'
            
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = str(bbox_points).lstrip('[').rstrip(']')
            
            # Print details
            print('    {}:{}:{}'.format(segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))
        
        
        
        # Draw segmentation polygons on image
        html  = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(image_path, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(adjusted_width, adjusted_height)
        
        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(points, fill_color, stroke_color)
        
        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)
            
        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)
                
        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html
       
    def process_info(self):
        self.info = self.coco['info']
    
    def process_licenses(self):
        self.licenses = self.coco['licenses']
    
    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            super_category = category['supercategory']
            
            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))

            # Add category to super_categories dict
            if super_category not in self.super_categories:
                self.super_categories[super_category] = {cat_id} # Create a new set with the category id
            else:
                self.super_categories[super_category] |= {cat_id} # Add category id to the set
                
    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image
                
    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)

In [ ]:
annotation_path = '../data/imgs/train/train.json'
image_dir = '../data/imgs/train/'

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_categories()

In [ ]:
html = coco_dataset.display_image(1)
IPython.display.HTML(html)

##**Setting Up Configurations**

Look through the code cell below and update any lines relevant to your custom dataset.
You may want to change:
*  NAME (might want to be more specific)
*  NUM_CLASSES (always 1 + the number of object categories you have)
*  IMAGE_MIN_DIM (if you have larger training images)
*  IMAGE_MAX_DIM (if you have larger training images)
*  STEPS_PER_EPOCH (if you want to train on more images each epoch)

In [ ]:
class CustomConfig(Config):
# Give the configuration a recognizable name    
    NAME = "skymaps"

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + weed

    # You can experiment with this number to see if it improves training
    STEPS_PER_EPOCH = 100 #should be 1000

    # This is how often validation is run. If you are using too much hard drive space
    # on saved models (in the MODEL_DIR), try making this value larger.
    VALIDATION_STEPS = 5

    # Don't exclude based on confidence. Since we have two classes
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    BACKBONE = "resnet50"

    # Input image resizing
    # Random crops of size 512x512
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_SCALE = 2.0

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64

    # Image mean (RGB)
    MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    TRAIN_ROIS_PER_IMAGE = 128

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 200

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 400

In [ ]:
# Code for Customdataset class. 
class CustomDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_custom(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "skymaps"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            #if class_id < 1:
            #   print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
            #   return
            if class_id >= 1:
              self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)
                
        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids                

##**Create Model**

In [ ]:
#LOAD MODEL. Create model in inference mode
MODEL_DIR = "/content/Mask_RCNN"
DEFAUTL_LOGS_DIR = os.path.join(MODEL_DIR, "logs")
WEIGHTS_PATH = "/content/Mask_RCNN/logs/skymaps20210727T2250/mask_rcnn_skymaps_0002.h5"
config = CustomConfig()
config.display()

In [ ]:
# Create Model
model = modellib.MaskRCNN(mode="training", config=config,model_dir=DEFAUTL_LOGS_DIR)
# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
# Exclude the last layers because they require a matching number of classes
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

##**Start Training**

In [ ]:

%reload_ext tensorboard
%tensorboard --logdir /content/Mask_RCNN/logs/skymaps20210727T2250/

In [ ]:
# Training dataset.
dataset_train = CustomDataset()
dataset_train.load_custom("/content/data/imgs/train/train.json", "/content/data/imgs/train/")
dataset_train.prepare()
print("Images: {}\nClasses: {}".format(len(dataset_train.image_ids), dataset_train.class_names))

# Validation dataset
dataset_val = CustomDataset()
dataset_val.load_custom("/content/data/imgs/validation/validation.json", "/content/data/imgs/validation/")
dataset_val.prepare()
print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

In [ ]:
!python -c 'import keras; print(keras.__version__)' #make sure 2.1.1 or mrcnn/model.py throws errors

In [ ]:
import time
from PIL import Image, ImageDraw, ImageFont, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
print("Training network heads")
start_train = time.time()
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=2,
                layers='heads')
end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
start_train = time.time()
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2,layers="all")
end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')

##**Detection**

In [ ]:
class InferenceConfig(CustomConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=DEFAUTL_LOGS_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
# Testing dataset.
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)
 
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)
 
visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

## **Evaluate Model**

In [ ]:
# Testing dataset.
dataset_test = CustomDataset()
dataset_test.load_custom("/content/data/imgs/test/test.json", "/content/data/imgs/test/")
dataset_test.prepare()
print("Images: {}\nClasses: {}".format(len(dataset_test.image_ids), dataset_test.class_names))
 
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_test.image_ids, 234)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
while True:pass